<a href="https://colab.research.google.com/github/saugnik/automated_email_sender/blob/main/crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install crewai langchain-groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 683.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
!pip show crewai

Name: crewai
Version: 0.30.11
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [ ]:
import os
os.environ["GROQ_API_KEY"]="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"

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            # api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192"
        )


In [ ]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
from datetime import datetime
from random import randint
from langchain.tools import tool

In [ ]:
import json
import os

import requests
from langchain.tools import tool

import os
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()

In [ ]:
search_tool.run("usb key price")

"The Feitian ePass K9 USB Security Key offers all the features expected from a security key at an affordable price. Feitian ePass K9 USB Security Key f ... The USB-C key is best suited for ... But its $286 asking price on Amazon (and even crazier $334 price directly from Corsair) feels like a bomb on your credit card bill. ... USB-C - USB-A - USB 3.2 Gen 2 (compatibel met PS4/PS5 ... Best security. 7. Kingston IronKey Keypad 200. View at Amazon. View at Amazon. View at Newegg. The best USB flash drive for security This ultra secure flash drive features USB 3.2 Gen 1 speeds and ... Offered in USB-A or USB-C models, in sizes ranging from 8 GB to 256 GB, this drive comes with a five-year warranty. Connector: USB 3.2 Gen 1 Type-A or Type C $92 at Amazon Posting field-leading write and read speeds of 249 MB/s and 415 MB/s, respectively, the PNY Pro Elite 256GB was the clear winner in our large file speed tests. The SanDisk Extreme GO 64GB was the only model that could get close to the PNY P

In [ ]:
class EmailAgents():
    def make_categorizer_agent(self):
        return Agent(
            role='Email Categorizer Agent',
            goal="""take in a email from a human that has emailed out company email address and categorize it \
            into one of the following categories: \
            Check Review-Classify the Emails.
            Other Things-Use this when we you get any Emails which not related to any type of review.
             """,
            backstory="""You are a master at understanding what a customer says when they write an email and are able to categorize it in a useful way""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Categorizer Agent"),
        )

    def make_researcher_agent(self):
        return Agent(
            role='Info Researcher Agent',
            goal="""take in a email from a human that has emailed out company email address and the category \
            that the categorizer agent gave it and decide what information you need to search for for the email writer to reply to \
            the email in a thoughtful and helpful way.
            If you DONT think a search will help just reply 'NO SEARCH NEEDED'
            If you dont find any useful info just reply 'NO USEFUL RESESARCH FOUND'
            otherwise reply with the info you found that is useful for the email writer
            """,
            backstory="""You are a master at understanding what review our email writer needs  to write a reply that \
            will help the customer""",
            llm=GROQ_LLM,
            verbose=True,
            max_iter=5,
            allow_delegation=False,
            memory=True,
            tools=[search_tool],
            step_callback=lambda x: print_agent_output(x,"Info Researcher Agent"),
        )

    def make_email_writer_agent(self):
        return Agent(
            role='Email Writer Agent',
            goal="""take in a email from a human that has emailed out company email address, the category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'Check Review' then classify the them into 3 parts.
            If the customer email is 'Other Thing' then try to search the answer and use it accrodingly.

                        You never make up information. that hasn't been provided by the researcher or in the email.
            Always sign off the emails in appropriate manner and from Gaurav the Resident Manager.
            """,
            backstory="""You are a master at synthesizing a variety of information and writing a helpful email \
            that will address the customer's issues and provide them with helpful information""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Writer Agent"),
        )



In [ ]:
class EmailTasks():
    # Define your tasks with descriptions and expected outputs
    def categorize_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and categorize into \
            one of the following categories:
            Positive Reviews: Thank the sender and encourage them to share their experience on social media. \
            Negative Reviews: Escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply. \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A single categtory for the type of email from the types ('Check Review', 'Other Thing') \
            eg:
            'review about the work' \
            """,
            output_file=f"email_category.txt",
            agent=categorizer_agent
            )

    def research_info_for_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and the category \
            provided and search the web to find info needed to respond to the email

            EMAIL CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email""",
            expected_output="""A set of bullet points of useful info for the email writer \
            or clear instructions that no useful material was found.""",
            context = [categorize_email],
            output_file=f"research_info.txt",
            agent=researcher_agent
            )

    def draft_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided, the category provided\
            and the info provided from the research specialist to write an email. \

            Write a simple, polite and too the point email which will respond to the customer's email. \
            If useful use the info provided from the research specialist in the email. \

            If no useful info was provided from the research specialist the answer politely but don't make up info. \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A well crafted email for the customer that addresses their issues and concerns""",
            context = [categorize_email, research_info_for_email],
            agent=email_writer_agent,
            output_file=f"draft_email.txt",
            )

In [ ]:
EMAIL = """HI there, \n
I am emailing to say that I had a wonderful Experience working with you all. \n

I really appreaciate what your staff did

Thanks,
Fahran
"""

EMAIL = """HI there, \n
I am emailing to say that the aera was not appropriate for my job.

I really hope you fix this next time.

Thanks,
Gaurav
"""

EMAIL = """HI there, \n
Provide me a better place for my file shooting

Thanks,
Saugnik
"""

In [ ]:
##
agents = EmailAgents()
tasks = EmailTasks()

## Agents
categorizer_agent = agents.make_categorizer_agent()
researcher_agent = agents.make_researcher_agent()
email_writer_agent = agents.make_email_writer_agent()

## Tasks
categorize_email = tasks.categorize_email(EMAIL)
research_info_for_email = tasks.research_info_for_email(EMAIL)
draft_email = tasks.draft_email(EMAIL)


In [ ]:
from crewai import Crew, Process

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[categorizer_agent, researcher_agent, email_writer_agent],
    tasks=[categorize_email, research_info_for_email, draft_email],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [ ]:
# Kick off the crew's work
results = crew.kickoff()


# Print the results
print("Crew Work Results:")
print(results)

# print(f"Categorize Email: {categorize_email.output}")
print(crew.usage_metrics)

 [DEBUG]: == Working Agent: Email Categorizer Agent
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the email provided and categorize into             one of the following categories:
            Positive Reviews: Thank the sender and encourage them to share their experience on social media.             Negative Reviews: Escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply. 
            EMAIL CONTENT:

 HI there, 

Provide me a better place for my file shooting

Thanks,
Saugnik
 


            Output a single cetgory only


> Entering new CrewAgentExecutor chain...
I understand the importance of this task and the expected format for the final answer. After conducting a comprehensive analysis of the email, I have categorized it as:

Thought: I now can give a great answer
Final Answer: Other Things

> Finished chain.
 [DEBUG]: == [Email Categorizer Agent] Task output: Other Things


 [DEBUG]: == Working Agen

In [ ]:
!cat /content/draft_email.txt

Dear Saugnik,

Thank you for reaching out to us! I'd be happy to help you find a better place for your file shooting.

There are several cloud storage options that you may find useful. You might consider trying out Google Drive, Dropbox, Microsoft OneDrive, iCloud, pCloud, or Box. Each of these options offers varying amounts of free storage, with features such as integration with specific apps, advanced security, and collaboration tools.

Here's a brief overview of what each option offers:

* Google Drive: 15 GB of free storage, integrates well with Google apps
* Dropbox: 2 GB of free storage, easy to use and share files
* Microsoft OneDrive: 5 GB of free storage, integrates well with Microsoft apps
* iCloud: 5 GB of free storage, integrates well with Apple devices
* pCloud: 10 GB of free storage, offers advanced security features
* Box: 10 GB of free storage, offers advanced collaboration features

You can consider factors such as storage capacity, pricing, security, and ease of use w

In [ ]:
!cat /content/email_category.txt

Other Things

In [ ]:
!cat /content/research_info.txt

Here are some popular cloud storage options that the customer may find useful:
* Google Drive: 15 GB of free storage, integrates well with Google apps
* Dropbox: 2 GB of free storage, easy to use and share files
* Microsoft OneDrive: 5 GB of free storage, integrates well with Microsoft apps
* iCloud: 5 GB of free storage, integrates well with Apple devices
* pCloud: 10 GB of free storage, offers advanced security features
* Box: 10 GB of free storage, offers advanced collaboration features

These options can be considered based on factors such as storage capacity, pricing, security, and ease of use.